Init all packages

In [ ]:
pip install nibabel numpy scipy matplotlib scikit-image torch torchvision pandas

GPU Available?

In [ ]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

Read Paths

In [ ]:
import csv
import pandas as pd
import numpy as np

rutas = pd.read_csv("d:/ADNI Data/directorios2.csv", header=None)
rutas.head()

Dicom to Nifti

In [ ]:
import os
import subprocess
import nibabel as nib
from scipy.ndimage import zoom

#Rutas
DCM2NIIX_PATH = r"D:/ADNI Data/dcm2niix_win/dcm2niix.exe"  
OUTPUT_NIFTI_DIR = r"D:/ADNI Data/Fixed/{0}/volumenes_nifti" 
OUTPUT_NPY_DIR = r"D:/ADNI Data/Fixed/{0}/volumenes_npy"
TARGET_SHAPE = (128, 128, 128)

for index, row in rutas.iterrows():
    Path_DIR = row[0]
    INPUT_DICOM_DIR = r"D:/ADNI Data/"+row[0] 
    
    print(INPUT_DICOM_DIR)
    OUTPUT_NIFTI_DIR =  OUTPUT_NIFTI_DIR.replace("{0}", Path_DIR)
    OUTPUT_NPY_DIR = OUTPUT_NPY_DIR.replace("{0}", Path_DIR)
    #Directorios
    os.makedirs(OUTPUT_NIFTI_DIR, exist_ok=True)
    os.makedirs(OUTPUT_NPY_DIR, exist_ok=True)

    # Recorrer carpetas
    for patient_folder in os.listdir(INPUT_DICOM_DIR):
        dcm_folder = os.path.join(INPUT_DICOM_DIR, patient_folder)
        if not os.path.isdir(dcm_folder): continue

        print(f"Procesando: {patient_folder}")

        # Convertir
        nifti_output_path = os.path.join(OUTPUT_NIFTI_DIR, patient_folder)
        os.makedirs(nifti_output_path, exist_ok=True)

        subprocess.run([
            DCM2NIIX_PATH,
            "-z", "y", 
            "-o", nifti_output_path,
            dcm_folder
        ])

        # Buscar el archivo
        nii_files = [f for f in os.listdir(nifti_output_path) if f.endswith('.nii.gz')]
        if not nii_files:
            print(f"No se pud convertir ningún NIfTI para {patient_folder}")
            continue

        nifti_path = os.path.join(nifti_output_path, nii_files[0])

        # Normalizar
        img = nib.load(nifti_path)
        data = img.get_fdata()
        data = (data - np.mean(data)) / np.std(data)

        # 3. Resize a 128x128x128
        factors = [t / s for t, s in zip(TARGET_SHAPE, data.shape)]
        resized = zoom(data, factors)

        # 4. Guardar numpy
        npy_path = os.path.join(OUTPUT_NPY_DIR, f"{patient_folder}.npy")
        np.save(npy_path, resized)

        print(f"Guardado: {npy_path}")

    
print("Conversión Finalizada.")

Instalar HD-BET

In [ ]:
pip install git+https://github.com/MIC-DKFZ/HD-BET.git

Extracción de cerebro

In [ ]:
import HD_BET
import torch
print("HD-BET cargado correctamente ✅")
print("CUDA disponible:", torch.cuda.is_available())


import os
import nibabel as nib
import numpy as np

for index, row in rutas.iterrows():
    Path_DIR = os.path.join(r"D:/ADNI Data/BET", row[0], "volumenes_nifti")
    for i in os.scandir(Path_DIR):
        for j in os.scandir(i.path):
            os.makedirs(i.path.replace("volumenes_nifti", "volumenes_npy"), exist_ok=True)
            if j.name.endswith(".nii.gz"):
                print(f"procesando {j.path}")
                nifti = nib.load(j.path)
                img_data = nifti.get_fdata()
                OUTPUT_NPY_DIR = j.path.replace("volumenes_nifti", "volumenes_npy").replace(".nii.gz", ".npy")
                np.save(OUTPUT_NPY_DIR, nifti)
print("Extracción finalizada.")

Crear CSV con rutas

In [ ]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split

rutaBet = r"D:\ADNI Data\BET\ADNI"
data = pd.DataFrame(columns=['Ruta', 'ID','Grupo'])
rutaAdni = r"D:/ADNI Data/T1_3D_3T_sagital_4_08_2025.csv"
adniCSV = pd.read_csv(rutaAdni, header= 0, sep=";")

for i in os.scandir(rutaBet):
    for j in os.scandir(i.path):
        for k in os.scandir(j.path):
            aux = os.path.join(k.path, "volumenes_nifti")
            for l in os.scandir(aux):
                IDAdni = l.name.split("\\")[-1]
                for m in os.scandir(l.path):
                    data = pd.concat([pd.DataFrame([[m.path,IDAdni,adniCSV[adniCSV['Image Data ID'] == IDAdni]['Group'].values[0]]], columns=data.columns), data], ignore_index=True)
                

data.to_csv(r"D:/ADNI Data/BETandGroups.csv", index=False)

CNdf = data[data['Grupo'] == 'CN']
MCIdf = data[data['Grupo'] == 'MCI']
ADdf = data[data['Grupo'] == 'AD']


df_train, df_test = train_test_split(data, test_size=0.25, random_state=42, stratify=data['Grupo'])
CNdf_train = df_train[df_train['Grupo'] == 'CN']
CNdf_test = df_test[df_test['Grupo'] == 'CN']   
MCIdf_train = df_train[df_train['Grupo'] == 'MCI']
MCIdf_test = df_test[df_test['Grupo'] == 'MCI']
ADdf_train = df_train[df_train['Grupo'] == 'AD']
ADdf_test = df_test[df_test['Grupo'] == 'AD']

print("CN Total:", len(CNdf),"CN train:", len(CNdf_train),"CN test:", len(CNdf_test)) 
print("MCI Total:", len(MCIdf),"MCI train:", len(MCIdf_train),"MCI test:", len(MCIdf_test)) 
print("AD Total:", len(ADdf),"AD train:", len(ADdf_train),"AD test:", len(ADdf_test))
